In [1]:
import numpy as np
import pandas as pd
import itertools

import copy
import sys, os
import datetime

from scipy.optimize import fsolve
from scipy.special import gamma


In [2]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__



In [3]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [4]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [5]:
def gen_call_arrival_time():
    return np.random.exponential(4)

In [6]:
def gen_call_priority():
    return int(np.random.uniform(0,1) < 0.25)

In [7]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [8]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [9]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [10]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [11]:
def get_ambulance_travel_time(distance, speed = 30):
    return distance/speed

In [12]:
class Ambulance(object):
    
    def __init__(self, base, speed = 30):
        
        #fix the ambulance's home base
        self.base = base
        
        #set the ambulance's travel speed
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = (-1, -1) # -1 if not assigned to any call, second one indicates priority
        
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self, speed, base = None):
        
        if base is None:
            base = self.base
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, base)
        travel_time = get_ambulance_travel_time(distance, speed)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = (-1, -1)
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
    def assign_to_call(self, call, time, index, speed):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance, speed)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = (call.call_index, call.priority) # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city, speed):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = np.inf
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance, speed)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [65]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, speed):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, 
                gen_call_arrival_time(), priority = gen_call_priority())
    
    i = timeline.shape[0]
    timeline.loc[i] = [call_index, call.priority, '', 0, time_arrival]
    if len(callList) >= M:
        # print("New call arrived. No more capacity. Reject call.")
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, call.priority, '', 7, time_arrival]
    
    else:
    
        # print("New call arrived. Add to call list.")
        callList[call_index] = call

        assign = -1
        index = 0

        nearest_distance = np.inf
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            # print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, call.priority, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign, speed)

        # else:
            # print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [14]:
def get_first(ambulance, callList, M, k):
    
    call_index = -1
    min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            if call_time < min_arrival_time:
                call_index = call_id
                min_arrival_time = call_time
        
    return call_index


In [15]:
def get_first_highpriority(ambulance, callList, M, k):
    
    # Assign based on FCFS within priority
    # assume only high and low, two levels or priority
    
    
    high_call_index = -1
    low_call_index = -1
    
    high_min_arrival_time = np.inf
    low_min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            
            if call.priority == 1:
                if call_time < high_min_arrival_time:
                    high_call_index = call_id
                    high_min_arrival_time = call_time
            
            else:
                if call_time < low_min_arrival_time:
                    low_call_index = call_id
                    low_min_arrival_time = call_time
            

    if high_call_index > -1:
        return high_call_index
    else:
        return low_call_index
    

In [16]:
def get_nearest(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_distance = np.inf
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [17]:
def get_nearest_threshold(ambulance, callList, M, k):
    # only serve unassigned calls within k distance from the ambulance
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance <= min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [18]:
def get_nearest_threshold_else_fcfs(ambulance, callList, M, k):
    # only serve unassigned calls within k distance from the ambulance
    # if no unassigned calls within the threshold, perform according to fcfs
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    min_arrival_time = np.inf
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
            elif distance > k:
                if call.arrival_time < min_arrival_time:
                    call_index = call_id
                    min_arrival_time = call.arrival_time
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [19]:
def get_nearest_highpriority(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_high_distance = np.inf
    min_low_distance = np.inf
    high_call_index = -1
    low_call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            
            if call.priority == 1:
                if distance < min_high_distance:
                    high_call_index = call_id
                    min_high_distance = distance
            else:
                if distance < min_low_distance:
                    low_call_index = call_id
                    min_low_distance = distance
    
    if high_call_index > -1:
        return high_call_index
    elif low_call_index > -1:
        return low_call_index
    else:
        return -1

In [89]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, timeline, policy, speed, time_threshold):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    
    if next_event_time == time_arrival:
        # print("New call arrived.")
        all_call = set(timeline['Call'])
        call_index = len(all_call) if -1 not in all_call else len(all_call)-1
        ambulanceList, callList, time_arrival, timeline = arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, speed)
        
    else:
        if ambulanceList[index_min].status == 1:
            # print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index, priority = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            # print("Now finish at-location treatment. Start going to hospital.")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city, speed)
            
        elif ambulanceList[index_min].status == 3:
            # print("Now reach hospital. Start transferring patient to hospital.")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            # print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                # print("Return to base.")
                ambulanceList[index_min].redployment(speed)
            else:
                # print("Call waiting. Assign to call in queue according to policy.")
                call_index = policy(ambulanceList[index_min], callList, M, time_threshold * speed/2)
                
                if call_index == -1:
                    # calls in callList have all been assigned with an ambulance, or exceed distance threshold
                    ambulanceList[index_min].redployment(speed)
                else:
                    i = timeline.shape[0]
                    call = callList[call_index]
                    timeline.loc[i] = [call_index, call.priority, index_min, 1, next_event_time]
                    ambulanceList[index_min].assign_to_call(call, next_event_time, index_min, speed)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = [-1, -1, index_min, 6, next_event_time]
            # print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline, next_event_time

In [21]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [22]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

In [23]:
def get_unreachable_calls(time, ambulanceList, callList, delta, speed = 10):
    
    unreach_calls = 0
    
    for ambulance in ambulanceList:
        if ambulance.status == 1:
            target_call = callList[ambulance.call]
            target_threshold = target_call.arrival_time + delta
            reach_outside = ambulance.endtime > target_threshold
            unreach_calls +=1
    
    return unreach_calls
            

In [24]:
def get_ambulance_current_loc(ambulance, time):
    
    ambulance_origin = ambulance.origin
    ambulance_destination = ambulance.destination
    
    ambulance_start = ambulance.time
    ambulance_end = ambulance.endtime
    
    current_time = time
    travel_ratio = (current_time - ambulance_start) / (ambulance_end - ambulance_start)
    
    current_location = ((ambulance_destination[0] - ambulance_origin[0]) * travel_ratio + ambulance_origin[0],
                        (ambulance_destination[1] - ambulance_origin[1]) * travel_ratio + ambulance_origin[1])
    
    return current_location

In [25]:
def get_uncovered_calls(time, ambulanceList, callList, delta, city, speed = 10):
    
    uncovered_base = []
    for base in city.base_location:
        cover_calls = 0
        for ambulance in ambulanceList:
            if ambulance.status == 0 or ambulance.status == 5:
                current_location = get_ambulance_current_loc(ambulance, time)
                distance_to_base = get_distance(current_location, base)
                time_to_base = get_ambulance_travel_time(distance_to_base, speed = speed)
                
                if time_to_base < delta:
                    cover_calls += 1
                
        
        if cover_calls == 0:
            uncovered_base.append(1)
        else:
            uncovered_base.append(0)
            
    return uncovered_base
            

In [26]:
def get_jobcount(timeline):
    timediff = np.append(np.diff(timeline['Timestamp']), 0)
    timeline['timediff'] = timediff
    
    n = timeline.shape[0]
    numCalls = np.zeros(n)
    
    count = 0
    for i in range(n):
        event = timeline.iloc[i]['Event']
        if event == 0:
            count += 1
        elif event == 5 or event == 7: 
            count -= 1
            
        if count <0:
            print("hi")
            
        numCalls[i] = count
        
    numCalls[-1] = numCalls[n-2]
    timeline['numCalls'] = numCalls
    return timeline

In [122]:
def get_jobs(timeline):
    total = max(timeline['Call'])
    
    arrival = np.zeros(total+1)*np.nan
    assign = np.zeros(total+1)*np.nan
    reach = np.zeros(total+1)*np.nan
    onsite = np.zeros(total+1)*np.nan
    transfer = np.zeros(total+1)*np.nan
    finish = np.zeros(total+1)*np.nan
    priority = np.zeros(total+1)
    
    for i in range(total + 1):
        c = timeline[timeline['Call'] == i]
        
        p = list(set(c['Priority']))[0]
        priority[i] = p
        
        n = c.shape[0]
        for index, row in c.iterrows():
            t = row['Timestamp']
            event = row['Event']
            if event == 0:
                arrival[i] = t
            elif event == 1:
                assign[i] = t if n > 1 else np.nan
            elif event == 2:
                reach[i] = t if n > 2 else np.nan
            elif event == 3:
                onsite[i] = t if n > 3 else np.nan
            elif event == 4:
                transfer[i] = t if n > 4 else np.nan
            elif event == 5:
                finish[i] = t if n > 5 else np.nan
            elif event == 7:
                finish[i] = t
#         print(n, arrival[i], assign[i], reach[i], onsite[i], transfer[i], finish[i])
        
    columns = ['priority', 'arrival_time', 'assigned_time', 'reach_patient', 'finish_onsite', 'reach_hospital', 'finish']
    data = list(zip(priority, arrival, assign, reach, onsite, transfer, finish))
    df = pd.DataFrame(data, columns=columns)
    
    df['waiting_time'] = df['assigned_time'] - df['arrival_time']
    df['total_time'] = df['finish'] - df['arrival_time']
    return df

In [142]:
def performance_metric(timeline, df, target, c = 4, verbose=True):
    
    result_dict = {}
    
    t = timeline.iloc[-1]['Timestamp']
    P = timeline.groupby('numCalls')['timediff'].sum() / t
    
    expectn = sum(P * P.index)
    try:
        expectw = sum(P[c+1:] * (P.index[c+1:]-c))
    except:
        expectw = sum(P[c+1:] * (P.index[c:]-c))
        
    utilization = (expectn - expectw) / c
    
    result_dict['totalCalls'] = df.shape[0]
    result_dict['utilization'] = utilization
    result_dict['expectNJobs'] = expectn
    result_dict['expectNQueue'] = expectw
    
    if verbose:
        print('Utilization:', utilization)
        print('Expected number of jobs in system:', expectn)
        print('Expected number of jobs in queue:', expectw)
    
    df_complete = df.dropna(axis=0)
    result_dict['expectedWaiting'] = np.mean(df_complete['waiting_time'])
    result_dict['expectedTotal'] = np.mean(df_complete['total_time'])
    result_dict['totalComplete'] = len(df_complete)
    
    if verbose:
        print('Expected time in queue:', np.mean(df_complete['waiting_time']))
        print('Expected time in system:', np.mean(df_complete['total_time']))
        print("Total completed patients: ",  len(df_complete))
    
    assigned = df[df['assigned_time'] > 0]
    count = 0
    for index, row in assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
    
    result_dict['totalAssigned'] = len(assigned)
    result_dict['totalUnreachable'] = count
    result_dict['rateUnreachable'] = count / df.shape[0]
    
    if verbose:
        print("Total assigned patients: ", len(assigned))
        print("Total unreachable calls:", count)
        print("Portion of patients that is unreachable:", count/df.shape[0])
    
    
    # Higher Priority
    highp = df[df['priority'] == 1]
    highp_complete = highp.dropna(axis=0)
    highp_assigned = highp[highp['assigned_time'] > 0]
    
    result_dict['totalHigh'] = len(highp)
    result_dict['totalCompleteHigh'] = len(highp_complete)
    result_dict['totalAssignedHigh'] = len(highp_assigned)
    
    if verbose:
        print("Total high priority patients: ",  len(highp))
        print("Total high priority patients completed: ",  len(highp_complete))
        print("Total high priority patients assigned: ",  len(highp_assigned))
    
    count = 0
    for index, row in highp_assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
    
    result_dict['totalUnreachableHigh'] = count
    result_dict['expectWaitingHigh'] = np.mean(highp_complete['waiting_time'])
    result_dict['expectTotalHigh'] = np.mean(highp_complete['total_time'])
    result_dict['rateUnreachableHigh'] = count/len(highp)
    
    if verbose:
        print("Total high priority unreachable calls:", count)
        print("Portion of high priority calls that is unreachable:", count/len(highp))
        print('Expected time in queue (high priority patients):', np.mean(highp_complete['waiting_time']))
        print('Expected time in system (high priority patients):', np.mean(highp_complete['total_time']))  
    
    # Lower Priority
    lowp = df[df['priority'] == 0]
    lowp_complete = lowp.dropna(axis=0)
    lowp_assigned = lowp[lowp['assigned_time'] > 0]

    result_dict['totalLow'] = len(lowp)
    result_dict['totalCompleteLow'] = len(lowp_complete)
    result_dict['totalAssignedLow'] = len(lowp_assigned)
    
    if verbose:
        print("Total low priority patients: ",  len(lowp))
        print("Total low priority patients completed: ",  len(lowp_complete))
        print("Total low priority patients assigned: ",  len(lowp_assigned))
    
    count = 0
    for index, row in lowp_assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
            
    result_dict['totalUnreachableLow'] = count
    result_dict['expectWaitingLow'] = np.mean(lowp_complete['waiting_time'])
    result_dict['expectTotalLow'] = np.mean(lowp_complete['total_time'])
    result_dict['rateUnreachableLow'] = count/len(lowp)
    
    if verbose:
        print("Total low priority unreachable calls:", count)
        print("Portion of low priority calls that is unreachable:", count/len(lowp))
        print('Expected time in queue (low priority patients):', np.mean(lowp_complete['waiting_time']))
        print('Expected time in system (low priority patients):', np.mean(lowp_complete['total_time'])) 
    
    
    return result_dict

# Set up the City

In [128]:
city_dimension = 150 # assume square
num_hospital = 2
num_base = 6
time_threshold = 9
call_list_threshold = 20
speed = 30

city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

# Heuristic Policies

In [129]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_first, speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [130]:
print((end - start).total_seconds())

15.544678


In [131]:
# enablePrint()

In [132]:
# time_arrival, ambulanceList, callList, timeline, time = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline, get_nearest_unassigned_threshold)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

Next Arrival:  5767.452348861964
Ambulance List Print: 
Status List:  [4, 2, 2, 2, 1, 0]
Origin List:  [(100.0, 75.0), (46.757328824726656, 72.6611633316614), (82.51572062397412, 21.640878993272317), (2.547159013046513, 100.24278686997461), (112.5, 50.0), (112.5, 100.0)]
Destination List:  [(100.0, 75.0), (46.757328824726656, 72.6611633316614), (82.51572062397412, 21.640878993272317), (2.547159013046513, 100.24278686997461), (25.21388089559251, 22.867139124511187), (112.5, 100.0)]
Event start time List:  [5760.7810989344, 5758.979961821191, 5756.598192362816, 5741.479718073862, 5758.2654745478985, 5714.1887097381405]
Event end time List:  [5763.243466689772, 5765.611424927091, 5772.50109503554, 5776.178297200601, 5762.079440547229, inf]
Call List Print: 
Status List:  [4]
Location List:  [(25.21388089559251, 22.867139124511187)]
Arrival time List:  [5758.2654745478985]


In [133]:
t1 = get_jobcount(timeline)

In [134]:
timeline

,Call,Priority,Ambulance,Event,Timestamp,timediff,numCalls
0,0,1,,0,0,0,1.0
1,0,1,3,1,0,0.289456,1.0
2,1,0,,0,0.289456,0,2.0
3,1,0,4,1,0.289456,0.366271,2.0
4,0,1,3,2,0.655727,0.82888,2.0
...,...,...,...,...,...,...,...
9158,1402,0,1,1,5757.76,0.505385,4.0
9159,1403,0,,0,5758.27,0,5.0
9160,1403,0,4,1,5758.27,0.714487,5.0
9161,1402,0,1,2,5758.98,1.80114,5.0


In [135]:
df = get_jobs(t1)

In [136]:
performance_metric(t1, df, time_threshold, c= num_base)

Utilization: 0.764159044851232
Expected number of jobs in system: 5.612491107001655
Expected number of jobs in queue: 1.027536837894263
Expected time in queue: 4.398762208271279
Expected time in system: 23.079729549978776
Total completed patients:  1399
Total assigned patients:  1403
Total unreachable calls: 369
Total high priority patients:  359
Total high priority patients completed:  358
Total high priority patients assigned:  358
Total high priority unreachable calls: 85
Percentage of high priority calls that is unreachable: 0.23676880222841226
Total low priority patients:  1045
Total low priority patients completed:  1041
Total low priority patients assigned:  1045
Total low priority unreachable calls: 284
Percentage of low priority calls that is unreachable: 0.27177033492822966


# FCFS with Priority

In [137]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [138]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_first_highpriority, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [143]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)

Utilization: 0.7848791048358236
Expected number of jobs in system: 5.815452173944262
Expected number of jobs in queue: 1.1061775449293207
Expected time in queue: 4.675543945107197
Expected time in system: 23.662586179012848
Total completed patients:  1413
Total assigned patients:  1416
Total unreachable calls: 352
Portion of patients that is unreachable: 0.24841213832039521
Total high priority patients:  345
Total high priority patients completed:  342
Total high priority patients assigned:  345
Total high priority unreachable calls: 23
Portion of high priority calls that is unreachable: 0.06666666666666667
Expected time in queue (high priority patients): 1.641858055557317
Expected time in system (high priority patients): 20.320028434227517
Total low priority patients:  1072
Total low priority patients completed:  1071
Total low priority patients assigned:  1071
Total low priority unreachable calls: 329
Portion of low priority calls that is unreachable: 0.3069029850746269
Expected time

{'totalCalls': 1417,
 'utilization': 0.7848791048358236,
 'expectNJobs': 5.815452173944262,
 'expectNQueue': 1.1061775449293207,
 'expectedWaiting': 4.675543945107197,
 'expectedTotal': 23.662586179012848,
 'totalComplete': 1413,
 'totalAssigned': 1416,
 'totalUnreachable': 352,
 'rateUnreachable': 0.24841213832039521,
 'totalHigh': 345,
 'totalCompleteHigh': 342,
 'totalAssignedHigh': 345,
 'totalUnreachableHigh': 23,
 'expectWaitingHigh': 1.641858055557317,
 'expectTotalHigh': 20.320028434227517,
 'rateUnreachableHigh': 0.06666666666666667,
 'totalLow': 1072,
 'totalCompleteLow': 1071,
 'totalAssignedLow': 1071,
 'totalUnreachableLow': 329,
 'expectWaitingLow': 5.644283977064293,
 'expectTotalLow': 24.729957559700633,
 'rateUnreachableLow': 0.3069029850746269}

# Neareset

In [119]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [120]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest, speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [123]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)


Utilization: 0.7797305131390476
Expected number of jobs in system: 6.064842431685111
Expected number of jobs in queue: 1.3864593528508256
Expected time in queue: 5.769271094475757
Expected time in system: 24.58834488287867
Total completed patients:  1420
Total assigned patients:  1420
Total unreachable calls: 258


,priority,arrival_time,assigned_time,reach_patient,finish_onsite,reach_hospital,finish,waiting_time,total_time
0,0.0,0.000000,0.000000,1.903843,10.891445,13.862758,16.393088,0.000000,16.393088
1,0.0,1.597693,1.597693,3.283036,6.134797,8.337143,10.778440,0.000000,9.180746
2,0.0,2.132882,2.132882,2.795120,13.465914,15.373538,17.743785,0.000000,15.610903
3,1.0,6.146588,6.146588,8.212231,11.735625,14.217935,16.642217,0.000000,10.495629
4,0.0,7.136632,7.136632,9.826562,11.964370,12.513759,15.022740,0.000000,7.886108
...,...,...,...,...,...,...,...,...,...
1416,0.0,5727.966963,5729.595405,5730.897170,5737.546373,5738.848138,5741.348000,1.628441,13.381037
1417,0.0,5737.915720,5737.915720,5739.445102,5742.779949,5745.404534,5747.737277,0.000000,9.821557
1418,0.0,5738.976158,5738.976158,5741.649722,5760.588601,NaN,NaN,0.000000,NaN
1419,0.0,5744.454830,5744.454830,5747.178426,5752.153174,5753.968935,5756.458416,0.000000,12.003585


# Nearest with High Priority

In [48]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [ ]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest_highpriority, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 

In [ ]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)

# Threshold supplemented with FCFS

In [ ]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [ ]:
# blockPrint()

start = datetime.datetime.now()
# unreach_callList = []
# uncovered_baseList = []
time = 0
horizon = 60*24*4
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(time_arrival, ambulanceList, callList, city, call_list_threshold, timeline, get_nearest_threshold_else_fcfs, 
                   speed, time_threshold)
    # print(len(callList))
    
    # unreach_call = get_unreachable_calls(time, ambulanceList, callList, time_threshold)
    # uncovered_base = get_uncovered_calls(time, ambulanceList, callList, time_threshold, city)
    
    # unreach_callList.append(unreach_call)
    # uncovered_baseList.append(uncovered_base)

end = datetime.datetime.now()

    
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, get_nearest_threshold, get_nearest_threshold_else_fcfs, 


In [ ]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
performance_metric(timeline, df, time_threshold, c= num_base)